In [85]:
from binary_string import binary as secret_message
from qunetsim import Host, Network, Logger, Qubit

import random


In [86]:


IS_EPR = '1'
IS_DATA = '0'

DATA_FRAME = 8

# The dataframe length should divide the length of the secret string
assert len(secret_message) % DATA_FRAME == 0

EPR_FRAME = 4


cur_location = 0
p = 0.75



In [87]:

def dense_encode(q: Qubit, bits: str):
    """
    Assumptions: - Qubit *q* is entangled with another qubit q' which resides at receiver
                 - Think of dense_encode as an optical device at the sender side, where each
                  qubit has to pass through the optical device

    Parameters
    ----------
    bits : str
        A two-bit string

    Returns
    -------
    Qubit
        The encoded qubit
    """

    # Depending on the 2 bit incoming message, determine the encoding
    # on qubit *q*.

    # TODO: Apply the superdense coding logic to qubit *q*
    if (str[0]=='0' and str[1]=='0'):
        q = q.I
    elif (str[0]=='0' and str[1]=='1'):
        q = q.X
    elif(str[0]=='1' and str[1]=='0'):
        q = q.Y
    elif(str[0]=='1' and str[1]=='1'):
        q = q.Z

    return q


In [88]:


def dense_decode(stored_epr_half: Qubit, received_qubit: Qubit) -> str:
    """
    Decode the two bit message encoded in *stored_epr_half* and *received_qubit*.

    Parameters
    ----------
    stored_epr_half : Qubit
        One half of the EPR pair for superdense coding
    received_qubit : Qubit
        One half of the EPR pair for superdense coding, received from sender

    Returns
    -------
    str
        Two bit message
    """
    meas = '00'

    # TODO: Apply the superdense decoding scheme to the two input qubits
    #       and return a string of the binary output

    stored_epr_half.m

    return meas


In [89]:


def encode_qubit(q: Qubit, bit: str) -> Qubit:
    """Encode the qubit *q* with bit *bit*.

    Parameters
    ----------
    q : Qubit
        The qubit to encode
    bit : str
        The bit to encode

    Returns
    -------
    Qubit
        The encoded qubit


    Raises
    ------
    Exception
        Input must be 0 or 1. Raise an exception otherwise.
    """

    # TODO: Based on the input bit, perform the respective encoding on
    #       the input qubit
    if (bit == '1'):
        q = q.X

    return q


In [90]:


def decode_qubit(q: Qubit) -> str:
    """Summary

    Parameters
    ----------
    q : Qubit
        The qubit to decode

    Returns
    -------
    str
        One bit with the qubit measurement result.
    """
    meas = q.measure()
    return str(meas)


In [91]:


def get_next_message() -> str:
    """
    With some probability, retreive *DATA_FRAME* bits of the message to transmit.
    When there are no more bits to transmit False is returned.

    Returns
    -------
    str
        A 1 or 2 bit message with probability *p*, -1 with *1 - p*, or False.
    """
    global cur_location
    if len(secret_message) == cur_location:
        return False

    should_send = random.random() <= p
    if should_send:
        msg = secret_message[cur_location: cur_location + DATA_FRAME]
        cur_location += DATA_FRAME
        return msg
    return -1



In [92]:

def decode_secret_message(binary_message: str):
    """
    Decode the ASCII values of *binary_message*.

    Parameters
    ----------
    binary_message : str
        The binary string to decode.
    """
    binary_int = int(binary_message, 2)
    byte_number = binary_int.bit_length() + 7 // 8
    binary_array = binary_int.to_bytes(byte_number, "big")
    ascii_text = binary_array.decode()
    print(f'Secret message:\n{ascii_text}')



In [93]:

def sender_protocol(host, receiver):
    cur_message = get_next_message()

    epr_pairs = []

    while cur_message:
        leading_qubit = Qubit(host)

        # Hint: Refer to the constants above for how to transmit the frames
        # Hint: Use the `await_ack=True` flag when sending qubits to keep
        #       the sender and receiver in sync
        if cur_message == -1:
            # TODO: Fill in the logic for when there is no message to send
            # Hint: You can use host.add_epr(receiver, qubit) to store EPR halfs
            #       after generating them
            leading_qubit = leading_qubit.X
            host.send_qubit(receiver, leading_qubit)   

            for i in range(EPR_FRAME):
                epr_id, ack_arrived = host.send_epr(receiver, await_ack=True)

                if ack_arrived:
                    # Receiver got the EPR pair and ACK came back
                    # safe to use the EPR pair.
                    epr_pairs.append(host.get_epr(receiver, q_id=epr_id))
            

        else:
            # TODO: Fill in the logic for when there is a message to send
            # Hint: You can use host.shares_epr(receiver) to determine how the
            #       message  should be encoded.
            # Hint: Use the encoding methods from above.
            leading_qubit = leading_qubit.X
            host.send_qubit(receiver, leading_qubit)

            while(cur_message.len>=2 and epr_pairs.count()>0):
                q = epr_pairs.pop()
                q = dense_encode(q, cur_message[:2])
                cur_message = cur_message[2:]
            while(cur_message.len>0):
                q = Qubit(host)
                q = encode_qubit(q,cur_message[0])
                cur_message = cur_message[1:]
            host.send_qubit(receiver, q)
        

        cur_message = get_next_message()



In [94]:

def receiver_protocol(host, sender):
    binary_message = ''
    epr_pairs_receiver = []
    while True:
        received_qubit = host.get_data_qubit(sender, wait=10)
        if received_qubit is None:
            break

        # TODO: Retreive the header bit
        header_bit = received_qubit.measure()
        if header_bit == IS_EPR:
            # TODO: Fill in the logic for what to do when the header qubit
            #       indicates EPR qubits arriving. Hint: EPR_FRAME defines
            #       how many EPR pair halves will arrive.

            for _ in range(EPR_FRAME):
                # Waits 5 seconds for the EPR to arrive.
                epr_pairs_receiver.append(host.get_epr(sender, wait=5))
                
        else:
            # TODO: Fill in the logic for what to do when the header qubit
            #       indicates data is arriving.
            # Hint: You can use host.shares_epr(sender) to determine how the
            #       message should be decoded
                if (epr_pairs_receiver.count > 0):
                   q_got = host.get_data_qubit(sender, wait=10)
                   q_had = epr_pairs_receiver.pop()
                   binary_message + dense_decode(q_had, q_got)
                else:
                   q = host.get_data_qubit(sender, wait=10)  
                   binary_message + decode_qubit(q)

                pass

    decode_secret_message(binary_message)



In [95]:

def main():
    network = Network.get_instance()
    network.start()

    host_A = Host('A')
    host_A.add_connection('B')
    host_A.start()
    host_B = Host('B')
    host_B.add_connection('A')
    host_B.start()

    network.add_hosts([host_A, host_B])

    t1 = host_A.run_protocol(sender_protocol, ('B',))
    t2 = host_B.run_protocol(receiver_protocol, ('A',), blocking=True)

    network.stop(True)

    pass

if __name__ == '__main__':
    main()


Exception in thread Thread-22 (sender_protocol):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_26267/3626282035.py", line 34, in sender_protocol
  File "/home/robi/.local/lib/python3.10/site-packages/qunetsim/components/host.py", line 1147, in send_qubit
    q.blocked = True
AttributeError: 'method' object has no attribute 'blocked'


Exception in thread Thread-23 (receiver_protocol):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_26267/4204331651.py", line 35, in receiver_protocol
  File "/tmp/ipykernel_26267/4099855621.py", line 10, in decode_secret_message
ValueError: invalid literal for int() with base 2: ''


In [9]:
meas = '00'
print(str("1") + str("0"))

10
